In [15]:
import pandas as pd
from datetime import date
import holidays
import numpy as np

In [16]:
#cargamos los datos
datos = pd.read_excel('../rawData/APARCAMIENTOS-DICIEMBRE-2019.xlsx')
datos.drop(columns='GarageNo', inplace=True) #eliminamos la columna de 'GarageNo' ya que no la usamos 

In [17]:
datos

,APARCAMIENTO,EventTime,EventDesc
0,VEGUETA,2022-12-01 00:11:30.000,SALIDA
1,VEGUETA,2022-12-01 00:13:46.000,SALIDA
2,VEGUETA,2022-12-01 00:24:56.000,ENTRADA
3,VEGUETA,2022-12-01 00:31:32.000,SALIDA
4,VEGUETA,2022-12-01 00:33:20.000,SALIDA
...,...,...,...
321765,SAN BERNARDO,2022-12-17 13:28:00.440,ENTRADA
321766,SAN BERNARDO,2022-12-24 07:04:03.710,SALIDA
321767,SAN BERNARDO,2022-12-24 10:21:42.310,ENTRADA
321768,SAN BERNARDO,2022-12-31 07:13:09.310,SALIDA


In [18]:
x, y = datos.shape #Obtenemos el shape de los datos
lugar = None #Creamos la varaibale lugar a None
res = {}

for l in datos['APARCAMIENTO'].unique():
    res[l] = []

#agrupamos por lugar los registros
for i in range(x):
    #Obtenemos el 'APARCAMIENTO' si no lo tenos ya
    if lugar == None or lugar != datos.iloc[i,0]:
        lugar = datos.iloc[i,0]
    month = datos.iloc[i,1].to_pydatetime().month
    year = datos.iloc[i,1].to_pydatetime().year
    res[lugar].append([datos.iloc[i,1].to_pydatetime(), datos.iloc[i,2]])

In [19]:
#usando los registros agrupados por lugar y los agrupamos por lugar y tipo
registros = {}
for l in datos['APARCAMIENTO'].unique():
    registros[l] = []

    E = []
    S = []
    for i in res[l]:
        if i[1].casefold() == 'SALIDA'.casefold():
            S.append(i)

        if i[1].casefold() == 'ENTRADA'.casefold():
            E.append(i)

        registros[l].append(S)
        registros[l].append(E)


In [20]:
#creamos el dicionario newData
newData = {}
for l in datos['APARCAMIENTO'].unique():
    newData[l] = {}
    for d in range(1,32):
        newData[l][d] = {}
        for h in range(0,24):
            newData[l][d][h] = {'SALIDAS':[date(year,month,d), 0],
                              'ENTRADAS':[date(year,month,d), 0]}

In [21]:
#rellenamos newData con los registros
for l in datos['APARCAMIENTO'].unique():
    #salida
    hora = None
    dia = None
    for i in registros[l][0]:
        if dia == None or dia != i[0].day:
            dia = i[0].day

        if hora == None or hora != i[0].hour:
            hora = i[0].hour

        newData[l][dia][hora]['SALIDAS'][1] = newData[l][dia][hora]['SALIDAS'][1]+1

    #entrada
    hora = None
    dia = None
    for i in registros[l][1]:
        if dia == None or dia != i[0].day:
            dia = i[0].day

        if hora == None or hora != i[0].hour:
            hora = i[0].hour

        newData[l][dia][hora]['ENTRADAS'][1] = newData[l][dia][hora]['ENTRADAS'][1]+1


In [22]:
festivos = holidays.Spain(subdiv='CN') # obtenemos los festivos de canarias
weekday = ['L','M','X','J','V','S','D'] # iniciales de los dias de la semana
noSchoolday = [date(2019,12,6), date(2019,12,9), date(2019,12,23), date(2019,12,24), date(2019,12,25), date(2019,12,26), date(2019,12,27), date(2019,12,30), date(2019,12,31),date(2022,12,6), date(2022,12,7), date(2022,12,8), date(2022,12,23), date(2022,12,26), date(2022,12,27), date(2022,12,28), date(2022,12,29), date(2022,12,30)]
df = pd.DataFrame(columns=['LOCALIZATION','DAY','MONTH','YEAR','WEEKDAY','SCHOOLDAY','HOLIDAY','HOUR','TEMPERATURE','HUMIDITY','CHECK IN', 'CHECK OUT']) # dataframe

#rellemanos el df con los nuevos datos
for lugar in newData.keys():
    for dia in newData[lugar].keys():
        for hora in newData[lugar][dia].keys():
            df = pd.concat([pd.DataFrame([[lugar, dia,month,year,date(year,month,dia).weekday(),date(year,month,dia) not in noSchoolday,date(year,month,dia) in festivos ,hora,np.nan,np.nan, newData[lugar][dia][hora]['ENTRADAS'][1], newData[lugar][dia][hora]['SALIDAS'][1]]], columns=df.columns), df], ignore_index=True)

C:\Users\madre\AppData\Local\Temp\ipykernel_53792\2003291628.py:10: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([pd.DataFrame([[lugar, dia,month,year,date(year,month,dia).weekday(),date(year,month,dia) not in noSchoolday,date(year,month,dia) in festivos ,hora,np.nan,np.nan, newData[lugar][dia][hora]['ENTRADAS'][1], newData[lugar][dia][hora]['SALIDAS'][1]]], columns=df.columns), df], ignore_index=True)


In [23]:
clima = pd.read_csv('../csv/CLIMA-DICIEMBRE-2019.csv') #obtenemos los datos del clima
#rellenamos los campos de 'TEMPERATURE' Y 'HUMIDITY' S
for i in range(len(clima)):
    index = df.loc[(df['HOUR']==clima.loc[i, 'Hora']) & (df['DAY']==clima.loc[i, 'Dia'])].index.tolist()
    df.loc[index,'TEMPERATURE'] = clima.loc[i, 'Tem']
    df.loc[index,'HUMIDITY'] = clima.loc[i, 'Hum']
#si existen valores NAN rellenaremos con el metodo pad (propagar la última observación válida hacia adelante hasta la siguiente observación válida)
df[['TEMPERATURE','HUMIDITY']] = df[['TEMPERATURE','HUMIDITY']].fillna(method='pad')

In [24]:
#Eliminamos los datos del parking de SAN BERNARDO ya que estuvo cerrado
df = df.loc[(df['LOCALIZATION']!='SAN BERNARDO')]
df.reset_index(drop=True, inplace=True)

In [25]:
#Exportamos el csv
df.to_csv('../csv/APARCAMIENTOS-DICIEMBRE-2019.csv', index=False)

In [26]:
df

,LOCALIZATION,DAY,MONTH,YEAR,WEEKDAY,SCHOOLDAY,HOLIDAY,HOUR,TEMPERATURE,HUMIDITY,CHECK IN,CHECK OUT
0,SAN BERNARDO,31,12,2022,5,True,False,23,18,88,0,2
1,SAN BERNARDO,31,12,2022,5,True,False,22,18,88,0,1
2,SAN BERNARDO,31,12,2022,5,True,False,21,18,83,19,12
3,SAN BERNARDO,31,12,2022,5,True,False,20,19,78,25,31
4,SAN BERNARDO,31,12,2022,5,True,False,19,20,78,15,63
...,...,...,...,...,...,...,...,...,...,...,...,...
5203,VEGUETA,1,12,2022,3,True,False,4,19,60,4,0
5204,VEGUETA,1,12,2022,3,True,False,3,18,68,0,1
5205,VEGUETA,1,12,2022,3,True,False,2,19,64,0,1
5206,VEGUETA,1,12,2022,3,True,False,1,19,60,0,0
